# 🧠 Smart Coral Cell-Level Mixer (DCMP + 7034)
This notebook matches the most accurate per-cell coral values (L1–L4) from DCMP and 7034 for each match and alliance.

In [1]:

import pandas as pd
from itertools import combinations, product

# Load files
dcmp_df = pd.read_csv("2025 DCMP - Match Data.csv")
csv7034_df = pd.read_csv("7034.csv")

# Normalize DCMP
dcmp_df['team_number'] = dcmp_df['team_key'].str.extract(r'(\d+)$').astype(float)
dcmp_df['match_key'] = dcmp_df['match_number'].astype(str) + '_' + dcmp_df['comp_level']

# Normalize 7034
csv7034_df = csv7034_df.dropna(subset=['Match #'])
csv7034_df['match_number'] = csv7034_df['Match #'].astype(int)
csv7034_df['team_number'] = csv7034_df['Team #']
csv7034_df['match_key'] = csv7034_df['match_number'].astype(str) + '_qm'
csv7034_df['alliance'] = csv7034_df['Robot'].str[0].map({'R': 'red', 'B': 'blue'})


In [2]:

def extract_alliance_teams(df):
    alliances = {}
    for _, row in df.iterrows():
        key = f"{row['match_number']}_{row['comp_level']}"
        if key not in alliances:
            alliances[key] = {
                'red': [row['r1'], row['r2'], row['r3']],
                'blue': [row['b1'], row['b2'], row['b3']],
            }
    return alliances

match_alliances = extract_alliance_teams(dcmp_df)

def assign_alliance(row):
    key = row['match_key']
    team_key = row['team_key']
    alliance = match_alliances.get(key, {})
    if 'red' in alliance and team_key in alliance['red']:
        return 'red'
    elif 'blue' in alliance and team_key in alliance['blue']:
        return 'blue'
    return None

dcmp_df['alliance'] = dcmp_df.apply(assign_alliance, axis=1)


In [3]:

def find_best_cell_level_combo(dcmp_rows, csv_rows, target, prefix):
    team_data = {}
    for row in dcmp_rows + csv_rows:
        team_number = row['team_number']
        if team_number not in team_data:
            team_data[team_number] = {}
        if f'{prefix}_L1' in row:
            team_data[team_number]['dcmp'] = row
        else:
            team_data[team_number]['7034'] = row

    best_diff = float('inf')
    best_combo = None

    for team_set in combinations(team_data.keys(), 3):
        robot_mix_options = list(product(['dcmp', '7034'], repeat=4))
        all_mix_combos = product(robot_mix_options, repeat=3)

        for robot_cell_choices in all_mix_combos:
            combo_rows = []
            valid = True
            total = 0

            for team, mix in zip(team_set, robot_cell_choices):
                sources = team_data[team]
                row = {}
                for i, lvl in enumerate(['L1', 'L2', 'L3', 'L4']):
                    dcmp_col = f'{prefix}_L{lvl[-1]}'
                    alt_col = f'{prefix}L{lvl[-1]}'
                    source = mix[i]
                    if source in sources:
                        src_row = sources[source]
                        val = src_row.get(dcmp_col) if source == 'dcmp' else src_row.get(alt_col, 0)
                        row[dcmp_col] = val
                        total += val
                    else:
                        valid = False
                        break
                row['team_number'] = team
                if not valid:
                    break
                combo_rows.append(row)

            if valid and abs(total - target) < best_diff:
                best_diff = abs(total - target)
                best_combo = combo_rows

    return best_combo


In [4]:

results = []

for match_key in dcmp_df['match_key'].unique():
    for alliance in ['red', 'blue']:
        dcmp_rows = list(dcmp_df[(dcmp_df['match_key'] == match_key) & (dcmp_df['alliance'] == alliance)].to_dict('records'))
        csv_rows = list(csv7034_df[(csv7034_df['match_key'] == match_key) & (csv7034_df['alliance'] == alliance)].to_dict('records'))
        if not dcmp_rows:
            continue

        target_auto = next((r['ba_autoCoralCount'] for r in dcmp_rows if pd.notna(r.get('ba_autoCoralCount'))), 0)
        target_tele = next((r['ba_teleopCoralCount'] for r in dcmp_rows if pd.notna(r.get('ba_teleopCoralCount'))), 0)

        team_info = {r['team_number']: r for r in dcmp_rows}
        best_auto = find_best_cell_level_combo(dcmp_rows, csv_rows, target_auto, 'a') or []
        best_tele = find_best_cell_level_combo(dcmp_rows, csv_rows, target_tele, 't') or []

        merged = {}
        for row in best_auto:
            tn = row['team_number']
            if tn not in merged:
                if tn in team_info:
                    merged[tn] = team_info[tn].copy()
                else:
                    # fallback to 7034
                    fallback_row = next((r for r in csv_rows if r['team_number'] == tn), {})
                    merged[tn] = {
                        'match_key': match_key,
                        'alliance': alliance,
                        'team_number': tn,
                        'team_key': f"frc{int(tn)}"
                    }
                    merged[tn].update(fallback_row)
            for k, v in row.items():
                if k.startswith('a_'):
                    merged[tn][k] = v
            merged[tn]['selected_for_auto'] = 'auto'

        for row in best_tele:
            tn = row['team_number']
            if tn not in merged:
                if tn in team_info:
                    merged[tn] = team_info[tn].copy()
                else:
                    # fallback: use full 7034 row if available
                    fallback_row = next((r for r in csv_rows if r['team_number'] == tn), {})
                    merged[tn] = fallback_row.copy()
                    # Ensure required metadata is present
                    merged[tn]['match_key'] = match_key
                    merged[tn]['alliance'] = alliance
                    merged[tn]['team_number'] = tn
                    merged[tn]['team_key'] = merged[tn].get('team_key', f"frc{int(tn)}")

            for k, v in row.items():
                if k.startswith('t_'):
                    merged[tn][k] = v
            merged[tn]['selected_for_teleop'] = 'teleop'

        for row in merged.values():
            row['match_key'] = match_key
            row['alliance'] = alliance
            results.append(row)

df_final = pd.DataFrame(results)
df_final = df_final.sort_values(by=['match_key', 'alliance', 'team_number'])
df_final.to_csv("smart_cell_level_corals2.csv", index=False)
print("Done! Saved to smart_cell_level_corals2.csv")


Done! Saved to smart_cell_level_corals.csv


In [6]:
import pandas as pd

# Load the merged output
df = pd.read_csv("smart_cell_level_corals.csv")

# Format and sort
df['match_number'] = pd.to_numeric(df['match_number'], errors='coerce').fillna(-1).astype(int)
df['alliance'] = pd.Categorical(df['alliance'], categories=['red', 'blue'], ordered=True)

# Sort by match number, red then blue
df_sorted = df.sort_values(by=['match_number', 'alliance', 'team_number'])

# Save result
df_sorted.to_csv("smart_cell_level_corals_full_sorted.csv", index=False)
print("✅ Sorted and saved to smart_cell_level_corals_full_sorted.csv")


✅ Sorted and saved to smart_cell_level_corals_full_sorted.csv
